In [1]:
# executeme

using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
# executeme

#@time begin
using QuantumOptics

#https://juliapackages.com/p/einsum
using Einsum

# https://github.com/Jutho/TensorOperations.jl
using TensorOperations

using BenchmarkTools

using LinearAlgebra

#end

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [3]:
Nx = 5; Ny = 5; N=Nx*Ny; q = Nx; cut_off = q
PN = [0,1,2]
U = 2;
Ncut = cut_off

5

In [4]:
sp_basis = NLevelBasis(N)
sp_matrix = Hofstadter_SP(Nx,Ny,1/q, 0);

In [5]:
# executeme

function get_sp_op(sp_basis, sp_matrix)
  
    H = SparseOperator(sp_basis)

    for m in 1:N
        for n in 1:N
            H = H + sp_matrix[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

get_sp_op (generic function with 1 method)

In [6]:
H1 = get_sp_op(sp_basis, sp_matrix);

In [7]:
#check op. form
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [8]:
# executeme

function get_sub_states(sp_op, cut_off)
    
    E0, states0 = eigenstates(dense(sp_op))
    states = states0[1:cut_off]
    
    return states
end

get_sub_states (generic function with 1 method)

In [9]:
sub_states = get_sub_states(H1, cut_off);

In [10]:
# executeme

function get_projector_op(states, basis)
    
    b_sub = SubspaceBasis(basis,states)
    P = projector(b_sub, basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_projector_op (generic function with 1 method)

In [11]:
b_sub, P, Pt = get_projector_op(sub_states, sp_basis);

In [12]:
# executeme

function get_subspace_op(sp_op, P, Pt)
    return P*sp_op*Pt
end

get_subspace_op (generic function with 1 method)

In [13]:
H1_sub = get_subspace_op(H1, P, Pt);

In [14]:
# executeme

function get_num_sub_list(sp_basis, P, Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(sp_basis, m, m)
        NMP = get_subspace_op(NM, P, Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list (generic function with 1 method)

In [15]:
num_sub_list = get_num_sub_list(sp_basis,P,Pt);

# Many-Body

In [16]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [17]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:21)

In [18]:
# executeme

function get_mb_op(basis_mb, basis, sp_op)
    
    Op_MB = SparseOperator(basis_mb)
    for i in 1:length(basis)
        for j in 1:length(basis)
            Op_MB = Op_MB + sp_op.data[i,j] * transition(basis_mb, i, j)
        end
    end
    
    return Op_MB
end

get_mb_op (generic function with 1 method)

In [19]:
H1_MB = get_mb_op(basis_mb, b_sub, H1_sub);

In [20]:
# H1_mb = manybodyoperator(basis_mb, H1_sub);

In [21]:
# executeme

function get_num_mb_list(basis_mb, basis, num_sub_list)
    num_mb_list = []
    for m in 1:N
        NMP = get_mb_op(basis_mb, basis, num_sub_list[m])
        push!(num_mb_list, NMP)
    end
    return num_mb_list
end

get_num_mb_list (generic function with 1 method)

In [22]:
num_mb_list = get_num_mb_list(basis_mb, b_sub, num_sub_list);

In [23]:
function get_hubbard_int(num_op_list, basis_mb, U)

    IT = SparseOperator(basis_mb)
    
    for m in 1:N
        IT = IT + U/2 * ( num_op_list[m] * num_op_list[m] - num_op_list[m] )
    end
    
    return IT
end

get_hubbard_int (generic function with 1 method)

In [24]:
H_Int_MB = get_hubbard_int(num_mb_list, basis_mb, U);

In [25]:

function get_hubbard_int2(basis, P, Pt)
    
    #@btime begin
    @time begin
    basis2 = basis ⊗ basis

    # interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i

    Vint = SparseOperator(basis2)

    for n in 1:N
        Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
    end

    Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

    Vint_mb = manybodyoperator(basis_mb, Vint_sub)
    end
    
    return Vint_mb
end

get_hubbard_int2 (generic function with 1 method)

In [26]:
H_Int_MB2 = get_hubbard_int2(sp_basis, P, Pt);

  0.271523 seconds (1.40 M allocations: 108.465 MiB, 7.38% gc time, 77.51% compilation time)


In [27]:
# executeme

function get_hubbard_int3(P, Pt, b_sub, cut_off)
    @time begin
   
    P1 = P.data
    P1t = Pt.data

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2_sub = b_sub ⊗ b_sub

    P4re = reshape(P4, cut_off^2, cut_off^2)

    Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re)
    #Vint_bsub2 = Operator(b2_sub,U/2*P4re)
    
    Vint_bsub2_mb = manybodyoperator(basis_mb, Vint_bsub2)
    end
    return Vint_bsub2_mb
end

get_hubbard_int3 (generic function with 1 method)

In [28]:
H_Int_MB3 = get_hubbard_int3(P, Pt, b_sub, cut_off);

  0.241042 seconds (1.31 M allocations: 87.571 MiB, 90.99% compilation time)


In [29]:
function get_hubbard_int4(P, Pt, b_sub, cut_off)
    
    bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off, PN)

    @time begin
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2cut = bcut ⊗ bcut

    P4re = reshape(P4, cut_off^2, cut_off^2)

    Vint_bsub2 = SparseOperator(b2cut, U/2*P4re)
        
    Vint_mb_cut = manybodyoperator(bcut_mb, Vint_bsub2)
    end
    
    return Vint_mb_cut
end

get_hubbard_int4 (generic function with 1 method)

In [30]:
H_Int_MB4 = get_hubbard_int4(P, Pt, b_sub, cut_off);

  0.143821 seconds (725.76 k allocations: 59.362 MiB, 21.86% gc time, 65.46% compilation time)


In [31]:
function get_hubbard_int5(P, Pt, b_sub, cut_off)
    
    bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off,PN)
    
    @time begin
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    Vint_mb_cut = SparseOperator(bcut_mb)
    for k in 1:Ncut
        for l in 1:Ncut
            for m in 1:Ncut
                for n in 1:Ncut
                    a1t = create(bcut_mb, k)
                    a2t = create(bcut_mb, l)
                    a2  = destroy(bcut_mb, m)      
                    a1  = destroy(bcut_mb, n)      
                    Vint_mb_cut += U/2*P4[k,l,m,n]*a1t*a2t*a2*a1
                end
            end
        end
    end
    end
    return Vint_mb_cut
end

get_hubbard_int5 (generic function with 1 method)

In [32]:
H_Int_MB5 = get_hubbard_int5(P, Pt, b_sub, cut_off);

  0.118220 seconds (124.72 k allocations: 11.323 MiB, 96.20% compilation time)


In [33]:
function manybodyoperator_2_Hubbard(basis::ManyBodyBasis, op::SparseOpType)
    N = length(basis)
    S = length(basis.onebodybasis)
    result = SparseOperator(basis)
    occupations = basis.occupations
    rows = QuantumOpticsBase.rowvals(op.data)
    values = QuantumOpticsBase.nonzeros(op.data)
    @inbounds for column=1:S^2, j in QuantumOpticsBase.nzrange(op.data, column)
        row = rows[j]
        value = values[j]
        index = Tuple(CartesianIndices((S, S, S, S))[(column-1)*S^2 + row])
        for m=1:N, n=1:N
            # println("row:", row, " column:"column, ind_left)
            C = QuantumOpticsBase.coefficient(occupations[m], occupations[n], index[1:2], index[3:4])
            if C!=0.
                result.data[m,n] += C*value
            end
        end
    end
    return result
end

manybodyoperator_2_Hubbard (generic function with 1 method)

In [34]:
isapprox( H_Int_MB2, H_Int_MB3 )

true

In [35]:
isapprox( H_Int_MB4, H_Int_MB5 )

true

In [36]:
isapprox( H_Int_MB4.data, H_Int_MB5.data )

true

In [37]:
isapprox( H_Int_MB3.data , H_Int_MB4.data )

true

In [38]:
H_MB = H1_MB + H_Int_MB
H_MB2 = H1_MB + H_Int_MB2
H_MB3 = H1_MB + H_Int_MB3
H_MB4 = H1_MB.data + H_Int_MB4.data;

In [39]:
bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off,PN)
H1cut = SparseOperator(bcut_mb)
H1cut.data = H1_MB.data
H1_MB

H_MB5 = H1cut + H_Int_MB5;

In [40]:
E = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E2 = eigenenergies(dense((H_MB2+dagger(H_MB2))/2))

E3 = eigenenergies(dense((H_MB3+dagger(H_MB3))/2))

E5 = eigenenergies(dense((H_MB5+dagger(H_MB5))/2))

print(E5[1:cut_off], "\n", 
      E2[1:cut_off], "\n", 
      E3[1:cut_off])

[-5.930026862496533, -5.930026862496532, -5.930026862496532, -5.930026862496527, -5.930026862496524]
[-5.9300268624965335, -5.93002686249653, -5.930026862496527, -5.930026862496527, -5.9300268624965256]
[-5.930026862496535, -5.9300268624965335, -5.930026862496529, -5.930026862496526, -5.930026862496525]

In [41]:
using LinearAlgebra
E4 = real(eigvals(Matrix(H_MB4)))


E0 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))[1:length(E)]

print(E0, "\n", E, " Incorrect one! \n", E2, "\n", E3, "\n", E4)

[-5.930812587456476, -5.930812587456471, -5.930812587456462, -5.930812587456459, -5.93081258745645, -5.81785420065732, -5.81785420065732, -5.8178542006573135, -5.81785420065731, -5.817854200657308, -5.759573954067759, -5.759573954067753, -5.7595739540677515, -5.759573954067751, -5.759573954067751, -3.868561021733692, -3.8685610217336817, -3.8685610217336786, -3.8685610217336777, -3.868561021733676, -3.8030181881052645]
[-7.530026862496533, -7.530026862496531, -7.530026862496529, -7.530026862496526, -7.530026862496523, -7.372090091593086, -7.372090091593083, -7.372090091593083, -7.372090091593082, -7.3720900915930745, -7.2965709807706265, -7.296570980770625, -7.296570980770623, -7.29657098077062, -7.296570980770619, -3.766447989143375, -3.766447989143375, -3.766447989143373, -3.7664479891433724, -3.7664479891433684, 0.0] Incorrect one! 
[-5.9300268624965335, -5.93002686249653, -5.930026862496527, -5.930026862496527, -5.9300268624965256, -5.7720900915930855, -5.772090091593083, -5.772090